In [1]:
import warnings
import pandas  as pd 
from bs4 import BeautifulSoup
import re
import string
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
users = pd.read_csv('Users.csv')
post = pd.read_csv('Posts.csv')

In [3]:
users['_AboutMe'].fillna(0, inplace = True)
#users

In [4]:
users_1 = users[users['_AboutMe']!=0]
# users_1

In [5]:
%%time
pre_p_desc = [] # to store pre processed aboutme

for desc in users_1['_AboutMe']:

    soup = BeautifulSoup(desc, 'html.parser')     # web scrapping - to get the text out of html tags

    article = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', soup.text, flags=re.MULTILINE)    # removing hyperlinks 

    article = re.sub("\d+", "", article)
    
    article = article.replace('\n',' ')

#         article = article.translate(str.maketrans(' ', ' ', string.punctuation))

    pun = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '=', '{', '}', '[', ']', '|', '\\', ":", ";", '"', '<', ',', '>', '.', '?', '/','♦']
    for i in pun :      # replacing punctuations with whitespace
        article = article.replace(i,' ')

    article = article.lower()     # to lower case

    article = article.replace('i\'m','i am ')     # removing contractions - I and am are considered as stopwords but not I'm

    pre_p_desc.append(article)

users_1['AboutMe_C'] = pre_p_desc

C:\Users\Mona\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Mona\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Wall time: 4.11 s


C:\Users\Mona\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
users_1 = users_1[['_Id','_AboutMe','AboutMe_C']]
# users_1

In [7]:
print(len(users_1['AboutMe_C']))

20800


In [8]:
#tf-idf

In [9]:
%%time
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(users_1['AboutMe_C'])                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T

Wall time: 3.63 s


In [10]:
#print(pairwise_similarity[20799])

In [11]:
%%time
for i in range(pairwise_similarity.shape[1]):
    pairwise_similarity[i,i:(i+1)] = 0

C:\Users\Mona\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  


Wall time: 6min 28s


In [12]:
print(pairwise_similarity.shape)

(20800, 20800)


In [12]:
mx_val = pairwise_similarity.argmax(1)

In [13]:
mx_val

matrix([[ 9441],
        [ 3259],
        [ 3259],
        ...,
        [17086],
        [15351],
        [16115]])

In [14]:
users_1 = users_1.reset_index()

In [15]:
sim_user = [] 
ccc = 0 
for i in mx_val:
    ccc += 1
    sim_user.append(users_1['_Id'][i.item()])

In [16]:
users_1['most_sim_users'] = sim_user

In [17]:
users_1.head(15)

,index,_Id,_AboutMe,AboutMe_C,most_sim_users
0,0,-1,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",hi i am not really a person i am a backgro...,35592
1,1,1,\r\n\r\n<p>Developer at Stack Overflow focusin...,developer at stack overflow focusing on publi...,13425
2,2,2,<p>Developer on the Stack Overflow team. Find...,developer on the stack overflow team find me...,13425
3,3,3,<p>I'm a student interested in technology.</p>...,i am a student interested in technology also...,18781
4,4,4,"<p>I work with <a href=""https://olo.com"" rel=""...",i work with olo solving problems for people wh...,30
5,5,5,"\r\n\r\n<p>Programmer, elected SE moderator ♦ ...",programmer elected se moderator on ppcg i...,41808
6,6,6,<p>I'm a researcher in satellite remote sensin...,i am a researcher in satellite remote sensing...,42326
7,7,7,"<p><a href=""//stackoverflow.com"">Stack Overflo...",stack overflow moderator i bite but only whe...,18695
8,8,8,"<p>Stack Exchange employs me as a <a href=""htt...",stack exchange employs me as a community manag...,59356
9,9,9,"<p>Maintainer of the <a href=""http://www.jdom....",maintainer of the jdom open source java xml li...,44417
